Diego Toribio <br>
Professor Fred Fontaine <br>
EID-378 Finance <br>
Problem Set II: Interest Rates and Bonds <br>

In [4]:
import numpy as np
import math
from tabulate import tabulate

## Section 1

The following code will perform backsubstitution: if $ A $ is a lower triangular matrix with nonzero elements on the diagonal, this will solve $ y = Ax $. Here, $ n $ is the length of the $ x,y $ vectors $( A $ is $ n \times n $):

```python
import numpy as np

x = np.zeros(n)

for i in range(0, n):
    tmp = y[i]
    for j in range(0, i):
        tmp -= x[j] * A[i, j]

    x[i] = tmp / A[i, i]
```

Feel free to modify it, convert it to a function, etc. It does not do error checking, and is based on the premise that $ A[i, j] = 0 $ for $ j > i $ (the code will still run if not, it just won’t solve $ y = Ax $ correctly), and that $ A[i, i] \neq 0 $.

Write Python code to do the following. Assume spot rates $ r $ and forward rates $ f $ are in continuous-compounding form, for simplicity. Given $ N $ bonds with prescribed coupon rates (coupons payable semi-annually), face value, current prices, and maturities exactly $ 0.5m $ years from today (i.e., coinciding with the coupon payments), with $ m = 1, 2, \dots, N $. You can have these values stored in whatever form you prefer—in separate `numpy.array` (vectors) or all together in a dataframe; note the above code, however, assumes $ y $ and $ A $ are `numpy` arrays.

(a) Solve for the discount factors $ Z(0, 0.5m) $ and from that term structure, i.e., $ r(0, 0.5m) $. Specifically, set up the matrix and use backsubstitution to solve for the term structure.

(b) From the term structure, compute the forward rates $ f(0, 0.5m, 0.5m + 0.5) $ for $ 0 \leq m \leq N - 1 $.

(c) Now apply your code to the following problem, assuming face value $\$1000$ for each bond:

| Bond Name | A      | B      | C      | D      |
|-----------|--------|--------|--------|--------|
| Maturity  | 0.5    | 1      | 1.5    | 2      |
| Bond Price| 985.86 | 974.70 | 967.99 | 966.82 |
| Coupon    | 2%     | 3%     | 4%     | 5%     |

(d) Plot the term structure (as a continuous curve), and display the forward rates.


### 1.1 - 

#### Context

To help clarify the following definitions, let’s start with a simple hypothetical example:

*Imagine you purchase a bond for $\$1,000$ with an annual coupon rate of $3\%$. This means that if you hold the bond for one year, it will pay you $\$30$ in interest, so at the end of the year, you receive $\$1,030$.*
<br>


**Term Structure:**  
This is a way to describe how interest rates change over different time periods. It represents the relationship between various maturities (e.g., 6 months, 1 year, 1.5 years, etc.) and the corresponding interest rates.

- In our hypothetical, the term structure might indicate that for a 1-year term the interest rate is 3%. If you had bonds with different maturities, each could have a different rate, and the collection of these rates forms the term structure.

**Discount Factors:**  
Discount factors tell us how much a future payment is worth in today’s dollars. They represent the present value of $\$1$ to be received at a future time. When interest is compounded continuously, the discount factor for a future time $T$ and a continuously compounded spot rate $r$ is given by:
$$
Z(0,T) = e^{-rT}.
$$

For example, if $r = 0.03$ (or $3\%$) and $T = 1$ year, then:
$$
Z(0,1) = e^{-0.03} \approx 0.97.
$$

- In our hypothetical, this means that receiving $\$1$ in one year is equivalent to having about $\$0.97$ today.

**Spot Rates:**  
Spot rates are the interest rates associated with a single cash flow received at a specific future time, assuming continuous compounding. They tell us the effective annual interest rate that would grow a present value to a specific future value. Spot rates are derived from discount factors using the formula:
$$
r(0,T) = -\frac{1}{T}\ln\Bigl(Z(0,T)\Bigr).
$$

For example, if the one-year discount factor is $Z(0,1) \approx 0.97$, then for $T = 1$ year:
$$
r(0,1) = -\ln(0.97) \approx 0.0305,
$$
which is approximately $3.05\%$ per year.

- In our hypothetical, this means that if receiving $\$1$ in one year is equivalent to having about $\$0.97$ today, then the effective continuously compounded interest rate for that period is roughly $3.05\%$.


**Calibrating the Term Structure:**  
Calibrating the term structure means adjusting our model so that the theoretical bond prices match actual market prices. We do this by working backwards: using observed bond prices and their future cash flows (like coupon payments), we solve for the discount factors that reflect the current value of future payments. 

- **Setting Up the System:**  
    To calibrate the term structure, we build a system of equations where each bond’s price equals the sum of its future cash flows, each adjusted by a discount factor. We organize the data into a matrix as follows:

   - **Rows:** Each row represents one bond.
   - **Columns:** Each column corresponds to a cash flow time (e.g., 0.5 years, 1 year, etc.).
   - **Matrix Entries:** These are the cash flow amounts for each bond at the corresponding time (with the final cash flow including the face value).
   - **Right-Hand Side:** This is a vector of the bonds’ market prices.


#### Overview
In part (a), our goal is to calibrate the term structure by solving for the discount factors $ Z(0, 0.5m) $ from the observed bond prices. Once we have the discount factors, we can derive the continuously compounded spot rates $ r(0, 0.5m) $ using the relation:
$$
r(0,T) = -\frac{1}{T}\ln\Bigl(Z(0,T)\Bigr).
$$
<br>


In [7]:
def solve_lower_triangular(A, y):
    # solve lower triangular system using back-substitution
    n = A.shape[0]
    x = np.zeros(n)
    for i in range(n):
        tmp = y[i]
        for j in range(i):
            tmp -= A[i, j] * x[j]
        x[i] = tmp / A[i, i]
    return x

def build_system(prices, coupons, face_value):
    # build system matrix and price vector for bond cash flows
    N = len(prices)
    A = np.zeros((N, N))
    y = np.array(prices)
    for i in range(N):
        for j in range(i + 1):
            # include coupon payment; add face value on final cash flow
            A[i, j] = coupons[i][j] + (face_value if j == i else 0)
    return A, y

# define bond data: prices, coupon cash flows, and face value
prices = [95.0, 92.0, 90.0]
coupons = [
    [2.0],
    [2.5, 2.5],
    [3.0, 3.0, 3.0]
]
face_value = 100.0

# build system for cash flows and bond prices
A, y_vec = build_system(prices, coupons, face_value)

# solve for discount factors using back-substitution
discount_factors = solve_lower_triangular(A, y_vec)

# compute maturities corresponding to each discount factor
maturities = np.array([0.5 * (i+1) for i in range(len(prices))])

# convert discount factors to continuously compounded spot rates
spot_rates = -np.log(discount_factors) / maturities

# prepare and output results table
rows = []
for i in range(len(prices)):
    bond_name = f"{i+1}"
    df = format(discount_factors[i], ".2g")
    sr = format(spot_rates[i], ".2g")
    rows.append([bond_name, df, sr])

headers = ["Bond", "Discount Factor", "Spot Rate"]
print(tabulate(rows, headers=headers, tablefmt="pretty"))

+------+-----------------+-----------+
| Bond | Discount Factor | Spot Rate |
+------+-----------------+-----------+
|  1   |      0.93       |   0.14    |
|  2   |      0.87       |   0.13    |
|  3   |      0.82       |   0.13    |
+------+-----------------+-----------+


### 1.2 - Forward Rates from Discount Factors

#### Context

After calibrating the term structure in part (a), we obtained discount factors that represent the current value of \$1 to be received at various future times. These discount factors provide a snapshot of the market’s interest rates for different maturities.

Forward rates build on this information—they indicate the implied future interest rate for a specific time period. In other words, while spot rates tell you the interest rate for a single cash flow at a future date, forward rates give you the rate for a period between two future dates.

#### Objective

The goal here is to compute the forward rates from the discount factors. Specifically, for the period between two consecutive cash flow times (for example, from 0.5 years to 1.0 years), we want to determine the continuously compounded forward rate that is consistent with the discount factors.

#### Approach

We use the relationship between discount factors and forward rates. Given the discount factors $Z(0,t)$ for times $t$ and $t+0.5$, the forward rate for the period $(t, t+0.5)$ is calculated as:

$$
f\bigl(0,\,t,\,t+0.5\bigr) \;=\; \frac{\ln Z(0,t) - \ln Z(0,t+0.5)}{0.5}.
$$

This formula works because when discount factors are expressed as $Z(0,t) = e^{-r(0,t)t}$, the difference in logarithms of the discount factors gives us the effective rate over the period.

In [12]:
def compute_forward_rates(discount_factors, maturities):
    # compute continuously compounded forward rates from discount factors
    forward_rates = []
    # for first period, use spot rate
    forward_rates.append(-np.log(discount_factors[0]) / maturities[0])
    for i in range(1, len(discount_factors)):
        dt = maturities[i] - maturities[i - 1]
        # compute forward rate between consecutive maturities
        f_rate = (np.log(discount_factors[i - 1]) - np.log(discount_factors[i])) / dt
        forward_rates.append(f_rate)
    return np.array(forward_rates)

# example discount factors and maturities
discount_factors = np.array([0.97, 0.94, 0.91])
maturities = np.array([0.5, 1.0, 1.5])

# compute forward rates
forward_rates = compute_forward_rates(discount_factors, maturities)

# prepare and display results table
rows = []
for i in range(len(forward_rates)):
    period = f"period {i+1}" if i > 0 else "period 1 (spot rate)"
    fr = format(forward_rates[i], ".2g")
    rows.append([period, fr])

headers = ["Period", "Forward Rate (continuous compounding)"]
print(tabulate(rows, headers=headers, tablefmt="pretty"))

+----------------------+---------------------------------------+
|        Period        | Forward Rate (continuous compounding) |
+----------------------+---------------------------------------+
| period 1 (spot rate) |                 0.061                 |
|       period 2       |                 0.063                 |
|       period 3       |                 0.065                 |
+----------------------+---------------------------------------+


## Section 2

Refer to the Python code below. It will compute the YTM from information about a bond (face value, price, and coupon rate).

```python
## From Mastering Python in Finance (with typo corrections by FF)
import scipy.optimize as optimize

def bond_ytm(price, FaceVal, T, coup, freq=2, guess=0.05):
    freq = float(freq)
    periods = T * freq
    coupon = coup / 100. * FaceVal / freq
    dt = [(i + 1) / freq for i in range(int(periods))]
    ytm_func = lambda y: sum([coupon / (1 + y / freq)**(freq * t) 
                              for t in dt]) + FaceVal / (1 + y / freq)**periods - price
    return optimize.newton(ytm_func, guess)
```

Write Python code that: given the term structure (annual) $ r_1(0,m) $, $ 1 \leq m \leq N $, and **annual** coupon rate (i.e., assume the coupons are paid annually) of a bond, will compute the price per $\$1$ face value and YTM of the bond.

Now take $ N = 10 $ and assume the Nelson-Siegel model with parameters $ \beta_0 = 0.02 $, $ \beta_1 = 0.02 $, $ \beta_2 = 0.20 $, $ \tau = 5 $:

$$
r_1(0,T) = \beta_0 + (\beta_1 + \beta_2)\frac{\tau_1}{T}\left(1 - e^{-T/\tau_1}\right) - \beta_2 e^{-T/\tau_1}
$$

Compute the price per $\$1$ face value and YTM for coupon rates $ 0\%, 1\%, \dots, 9\% $, and plot each curve.


## Section 3

The Macauley duration $ D_{mac} $ assuming annual compounding can be expressed as:

$$
D_{mac} = 1 + \frac{1}{y_1} + \frac{T\,(y_1 - c) - (1 + y_1)}{c\left[(1+y_1)^T - 1\right] + y_1}
$$

where $ y_1 $ is the YTM, $ c $ is the coupon rate, and $ T $ is time-to-maturity in years. As a hint to its behavior, regardless of $ c $:

$$
\lim_{T \to \infty} D_{mac} = 1 + \frac{1}{y_1}
$$

(a) Write a function in Python to compute $ D_{mac} $ from these three parameters.

(b) As an example, set $ y_1 = 10\% $ and graph duration as a function of time to maturity, up to $ T = 100 $ years, for $ c = 2\% $ and $ 10\% $ (superimposed) with a horizontal line indicating the limiting value $ D_{mac}(T = \infty) $. [The reason for going out to 100 years is to show the convergence.] This should replicate a graph in the Brandimarte text.

(c) Generate several plots to help us visualize how $ D_{mac} $ varies with these parameters. What you do is up to you. We don’t want 1,000 plots, and the plots should have reasonable values (the above was an exception—don’t take $ T > 30 $ years normally). Do whatever you think is reasonable to illustrate how $ D_{mac} $ varies with these parameters.


## Section 4

Companies $ A $ and $ B $ have been offered the following rates per year on a $\$1$ million, 5-year investment:

|           | Fixed Rate | Floating Rate |
|-----------|------------|---------------|
| **A**     | 8.8%       | LIBOR         |
| **B**     | 8.0%       | LIBOR         |

(a) Company $ A $ prefers a fixed-rate loan, and company $ B $ prefers a floating-rate loan. Bank $ X $ has been engaged as an intermediary for a swap. The swap should be equally attractive to each company, and the bank should earn 0.2% annually. Design the swap.

(b) Suppose instead company $ A $ were offered a fixed rate of 8.0% and company $ B $ a rate of 8.8%. If you repeat your calculation, you will find a problem with it, and neither company would actually engage in the transaction. Explain in *words*, based on the discussion in the lecture: why doesn’t a swap make sense here?